In [1]:
# Instalar las dependencias necesarias
!pip install rdflib pandas

# Importar las bibliotecas
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Ruta del archivo CSV en Google Drive
csv_file_path = '/content/drive/My Drive/mineduc-directorio-oficial.csv'

# Cargar el archivo CSV
data = pd.read_csv(csv_file_path, sep=';', encoding='utf-8')

# Crear un grafo RDF vacío
graph = Graph()

# Definir el prefijo para la URI base
namespace = Namespace("https://datosabiertos.mineduc.cl/directorio-de-establecimientos-educacionales/")
schema = Namespace("https://schema.org/")

# Vincular namespaces
graph.bind("mineduc", namespace)
graph.bind("schema", schema)

# Función para validar y convertir valores numéricos
def parse_float(value):
    try:
        # Reemplazar caracteres no numéricos si es necesario
        value = str(value).replace(',', '.').replace('°', '').strip()
        return float(value)
    except (ValueError, TypeError):
        return None

# Convertir los datos en triples RDF
for _, row in data.iterrows():
    # Crear URI para cada establecimiento basado en su RBD
    school_uri = URIRef(f"{namespace}{row['RBD']}")

    geo_uri = URIRef(f"{school_uri}/geo")

    # Declarar que el recurso es de tipo schema:School
    graph.add((school_uri, RDF.type, schema.School))

    # Agregar propiedades del esquema schema.org
    graph.add((school_uri, schema.name, Literal(row['NOM_RBD'], datatype="xsd:string")))

    # Agregar coordenadas geográficas
    geo_uri = URIRef(f"{school_uri}/geo")
    lat = parse_float(row['LATITUD'])
    lng = parse_float(row['LONGITUD'])
    if lat is not None and lng is not None:
        graph.add((school_uri, schema.geo, geo_uri))
        graph.add((geo_uri, RDF.type, schema.GeoCoordinates))
        graph.add((geo_uri, schema.latitude, Literal(lat, datatype="xsd:float")))
        graph.add((geo_uri, schema.longitude, Literal(lng, datatype="xsd:float")))

    # Agregar propiedades adicionales relacionadas al namespace mineduc
    graph.add((school_uri, namespace.nom_com_rbd, Literal(row['NOM_COM_RBD'], datatype="xsd:string")))
    graph.add((school_uri, namespace.nom_reg_rbd_a, Literal(row['NOM_REG_RBD_A'], datatype="xsd:string")))
    graph.add((school_uri, namespace.nom_com_rbd, Literal(row['NOM_COM_RBD'], datatype="xsd:string")))
    graph.add((school_uri, namespace.nom_deprov_rbd, Literal(row['NOM_DEPROV_RBD'], datatype="xsd:string")))
    graph.add((school_uri, namespace.rut_sostenedor, Literal(row['RUT_SOSTENEDOR'], datatype="xsd:string")))
    graph.add((school_uri, namespace.year, Literal(row['AGNO'], datatype="xsd:gYear")))
    graph.add((school_uri, namespace.rbd, Literal(row['RBD'], datatype="xsd:string")))
    graph.add((school_uri, namespace.dgv_rbd, Literal(row['DGV_RBD'], datatype="xsd:string")))
    graph.add((school_uri, namespace.p_juridica, Literal(row['P_JURIDICA'], datatype="xsd:string")))
    graph.add((school_uri, namespace.cod_reg_rbd, Literal(row['COD_REG_RBD'], datatype="xsd:string")))
    graph.add((school_uri, namespace.cod_pro_rbd, Literal(row['COD_PRO_RBD'], datatype="xsd:string")))
    graph.add((school_uri, namespace.cod_com_rbd, Literal(row['COD_COM_RBD'], datatype="xsd:string")))
    graph.add((school_uri, namespace.cod_deprov_rbd, Literal(row['COD_DEPROV_RBD'], datatype="xsd:string")))
    graph.add((school_uri, namespace.cod_depe, Literal(row['COD_DEPE'], datatype="xsd:string")))
    graph.add((school_uri, namespace.cod_depe2, Literal(row['COD_DEPE2'], datatype="xsd:string")))
    graph.add((school_uri, namespace.rural_rbd, Literal(row['RURAL_RBD'], datatype="xsd:boolean")))
    graph.add((school_uri, namespace.convenio_pie, Literal(row['CONVENIO_PIE'], datatype="xsd:boolean")))
    graph.add((school_uri, namespace.pace, Literal(row['PACE'], datatype="xsd:boolean")))
    graph.add((school_uri, namespace.mat_total, Literal(row['MAT_TOTAL'], datatype="xsd:integer")))
    graph.add((school_uri, namespace.matricula, Literal(row['MATRICULA'], datatype="xsd:integer")))
    graph.add((school_uri, namespace.estado_estab, Literal(row['ESTADO_ESTAB'], datatype="xsd:string")))
    graph.add((school_uri, namespace.ori_religiosa, Literal(row['ORI_RELIGIOSA'], datatype="xsd:string")))
    graph.add((school_uri, namespace.ori_otro_glosa, Literal(row['ORI_OTRO_GLOSA'], datatype="xsd:string")))
    graph.add((school_uri, namespace.pago_matricula, Literal(row['PAGO_MATRICULA'], datatype="xsd:boolean")))
    graph.add((school_uri, namespace.pago_mensual, Literal(row['PAGO_MENSUAL'], datatype="xsd:boolean")))

    # Agregar propiedades de especialidad (ens) y su matrícula
    for i in range(1, 12):
        ens_key = f"ENS_0{i}" if i < 10 else f"ENS_{i}"
        espe_key = f"ESPE_0{i}" if i < 10 else f"ESPE_{i}"
        if ens_key in row and not pd.isna(row[ens_key]):
            graph.add((school_uri, URIRef(f"{namespace}{ens_key.lower()}"), Literal(row[ens_key], datatype="xsd:string")))
        if espe_key in row and not pd.isna(row[espe_key]):
            graph.add((school_uri, URIRef(f"{namespace}{espe_key.lower()}"), Literal(row[espe_key], datatype="xsd:string")))

# Guardar el grafo RDF en formato Turtle en Google Drive
rdf_file_path = '/content/drive/My Drive/datosMineduc_23012025.ttl'
graph.serialize(destination=rdf_file_path, format='turtle')

print(f"Archivo RDF guardado en: {rdf_file_path}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 5.8 MB/s eta 0:00:00
Mounted at /content/drive
Archivo RDF guardado en: /content/drive/My Drive/datosMineduc_23012025.ttl


In [2]:
import json
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import XSD, RDF

# Definir namespaces
EX = Namespace("https://midas.minsal.cl/")
BASE = Namespace("https://midas.minsal.cl/farmacias_v2/")
SCHEMA = Namespace("https://schema.org/")

# Crear grafo RDF limpio
g = Graph()
g.bind("ex", EX)
g.bind("schema", SCHEMA)

# Ruta del archivo JSON cargado
json_file = "/content/drive/My Drive/minsal-farmacias.json"  # Ajustar según la ruta en tu sistema

# Función para validar y convertir coordenadas a float
def parse_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# Leer el archivo JSON
with open(json_file, encoding='utf-8') as file:
    data = json.load(file)
    for entry in data:
        # Crear URI para cada farmacia usando su local_id
        subject = URIRef(BASE + str(entry["local_id"]))

        # Declarar que el sujeto es del tipo schema:Pharmacy usando RDF.type
        g.add((subject, RDF.type, SCHEMA.Pharmacy))

        # Utilizar schema.org para los campos relacionados
        g.add((subject, SCHEMA.name, Literal(entry["local_nombre"], datatype=XSD.string)))
        g.add((subject, SCHEMA.address, Literal(entry["local_direccion"], datatype=XSD.string)))
        g.add((subject, SCHEMA.telephone, Literal(entry["local_telefono"], datatype=XSD.string)))

        # Validar y agregar geo-coordenadas si son válidas
        lat = parse_float(entry.get("local_lat"))
        lng = parse_float(entry.get("local_lng"))
        if lat is not None and lng is not None:
            geo_node = URIRef(f"{BASE}geo/{entry['local_id']}")
            g.add((subject, SCHEMA.geo, geo_node))
            g.add((geo_node, RDF.type, SCHEMA.GeoCoordinates))
            g.add((geo_node, SCHEMA.latitude, Literal(lat, datatype=XSD.float)))
            g.add((geo_node, SCHEMA.longitude, Literal(lng, datatype=XSD.float)))

        # Agregar horarios de apertura
        g.add((subject, SCHEMA.openingHours, Literal(
            f"{entry['funcionamiento_dia']} {entry['funcionamiento_hora_apertura']}-{entry['funcionamiento_hora_cierre']}",
            datatype=XSD.string)))

        # Agregar información adicional usando el esquema EX
        g.add((subject, EX.fecha, Literal(entry["fecha"], datatype=XSD.string)))
        g.add((subject, EX.local_id, Literal(entry["local_id"], datatype=XSD.string)))
        g.add((subject, EX.comuna, Literal(entry["comuna_nombre"], datatype=XSD.string)))
        g.add((subject, EX.localidad, Literal(entry["localidad_nombre"], datatype=XSD.string)))
        g.add((subject, EX.region_id, Literal(entry["fk_region"], datatype=XSD.string)))
        g.add((subject, EX.comuna_id, Literal(entry["fk_comuna"], datatype=XSD.string)))
        g.add((subject, EX.localidad_id, Literal(entry["fk_localidad"], datatype=XSD.string)))

# Guardar el grafo en formato RDF
output_file = "/content/drive/My Drive/minsal-farmacias-16012025-v5.ttl"
g.serialize(destination=output_file, format="turtle")

print(f"Archivo RDF generado: {output_file}")


Archivo RDF generado: /content/drive/My Drive/minsal-farmacias-16012025-v5.ttl
